In [1]:
from google.colab import files

select = files.upload()

Saving dataset.csv to dataset.csv


In [2]:
import pandas as pd

df = pd.read_csv("dataset.csv",encoding='latin')
df.columns = ['tweets','targets']

In [3]:
import re

def data_cleansing(tweet):  
    
    # removal of extra spaces
    re_pattern = re.compile(r'\s+')
    spaceRefined = tweet.str.replace(re_pattern, ' ')

    # removal of @name[mention]
    re_pattern = re.compile(r'@[\w\-]+')
    usernameRefined = spaceRefined.str.replace(re_pattern, '')

    # removal of links[https://abc.com]
    re_pattern =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = usernameRefined.str.replace(re_pattern, '')
    
    # removal of punctuations and numbers
    puncRefined = tweets.str.replace("[^a-zA-Z]", " ")
    # remove whitespace with a single space
    newtweet = puncRefined.str.replace(r'\s+', ' ')
    # remove leading and trailing whitespace
    newtweet = newtweet.str.replace(r'^\s+|\s+?$','')
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
    # removal of capitalization
    tweet_lower = newtweet.str.lower()

    return tweet_lower

refinedTweets = data_cleansing(df.tweets)   
df['refined'] = refinedTweets

In [4]:
print(df.columns)

Index(['tweets', 'targets', 'refined'], dtype='object')


In [5]:
print("Total examples of hate speech ", df.query('targets == 0').shape[0])
print("Total examples of non hate speech ", df.query('targets == 1').shape[0])

Total examples of hate speech  3358
Total examples of non hate speech  3850


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
df_tfidf = tfidf_vectorizer.fit_transform(df.refined)



unseenl = []
unseenl.append('tm bary bhenchod ho')
unseenl.append('randi k bachy')
unseenl.append('lakh lanat tery moh ty kisy ghasti dy bachya')



unseentfidf = tfidf_vectorizer.fit_transform(unseenl)

In [17]:
print("First row of dataset","'", df.refined[0],"'")
print("Tfidf Representation of row ")

First row of dataset ' bhaunk bhaunk ke pagal ho jaata ab tak '
Tfidf Representation of row 
(3, 15)


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

X = df_tfidf
y = df.targets.astype(int)

print()

X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)

In [19]:
print(X_train_tfidf.shape, X_test_tfidf.shape)
print(unseentfidf.shape)

(5766, 17994) (1442, 17994)
(3, 15)


In [ ]:
# Multilayer perceptron
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(activation='relu',learning_rate_init=0.01, solver='sgd').fit(X_train_tfidf, y_train)
y_preds = model.predict(X_test_tfidf)
report = classification_report( y_test, y_preds )
print(report)
acc = accuracy_score(y_test,y_preds)
print("MLP Classifier, Accuracy Score:" , acc)